## 10.1 분산을 기준으로 수치 특성 선택하기
수치형 특성 중에서 분산이 낮은 특성(즉 정보가 거의 없는 특성)을 삭제하고 싶습니다.

주어진 기준값보다 높은 분산을 가진 특성을 선택합니다.

In [3]:
from sklearn import datasets
from sklearn.feature_selection import VarianceThreshold

iris = datasets.load_iris()

features = iris.data
target = iris.target

# 기준값을 만듭니다.
thresholder = VarianceThreshold(threshold=.5)

# 기준값보다 높은 특성을 선택합니다.
features_high_variance = thresholder.fit_transform(features)

# 선택한 특성을 확인합니다.
features_high_variance[0:3]

array([[5.1, 1.4, 0.2],
       [4.9, 1.4, 0.2],
       [4.7, 1.3, 0.2]])

분산 기준 설정(variance thresholding, VT)는 가장 기본적인 특성 선택 방법 중 하나입니다. 이 방식은 분산이 높은 특성보다 분산이 낮은 특성이 효과적이거나 유용하지 않다는 아이디어에 기반합니다. VT는 먼저 각 특성의 분산을 계산합니다.

$$Var(x) = \frac{1}{n}\sum_{i=1}^n(x_i-\mu)^2$$

여기에서 $x$는 특성 벡터이고 $x_i$는 개별 특성값입니다. $\mu$는 특성의 평균값입니다. 그다음 분산이 기준값에 미치지 못하는 모든 특성을 삭제합니다.

VT를 사용할 때 두 가지를 기억해야 합니다. 첫째, 분산은 원점에 맞춰진 값이 아닙니다. 즉 특성의 제곱 단위입니다. 따라서 특성의 단위가 서로 다르면 VT가 작동하지 않습니다(예를 들어 한 특성은 년 단위이고 다른 특성은 원 단위인 경우). 둘째, 분산의 기준값을 수동으로 선택하기 때문에 어떤 값이 좋은지 판단할 수 있어야 합니다(또는 12장에서 소개하는 모델 선택 기법을 사용합니다). variances_ 속성에서 각 특성의 분산을 확인할 수 있습니다.

In [4]:
# 분산을 확인합니다.
thresholder.variances_

array([0.68112222, 0.18871289, 3.09550267, 0.57713289])

마지막으로 특성이 (평균이 0이고 단위 분산으로) 표준화되어 있으면 당연하지만 분산 기준 선택 방식은 올바르게 작동하지 않습니다.

In [5]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
features_std = scaler.fit_transform(features)

selector = VarianceThreshold()
selector.fit(features_std).variances_

array([1., 1., 1., 1.])

## 10.2 분산을 기준으로 이진 특성 선택하기
이진 범주형 특성(binary categorical feature)에서 분산이 낮은 특성(즉 적은 정보를 가진 특성)을 삭제하고 싶습니다.

Bernoulli random variable의 분산이 기준값 이상인 특성을 선택합니다.

In [6]:
from sklearn.feature_selection import VarianceThreshold

# 예제 특성 행렬을 만듭니다.
# 특성 0: 80%가 클래스 0
# 특성 1: 80%가 클래스 1
# 특성 2: 60%가 클래스 0, 40%는 클래스 1
features = [[0, 1, 0],
            [0, 1, 1],
            [0, 1, 0],
            [0, 1, 1],
            [1, 0, 0]]

# 분산을 기준으로 선택합니다.
thresholder = VarianceThreshold(threshold=(.75 * (1-.75)))
thresholder.fit_transform(features)

array([[0],
       [1],
       [0],
       [1],
       [0]])

수치형 특성과 마찬가질 정보가 많은 범주형 특성을 선택하는 한 가지 전략은 분산을 조사하는 것입니다. 이진 특성(즉 베르누이 확률 변수)의 분산은 다음과 같이 계산합니다.
$$Var(x) = p(1-p)$$

여기에서 $p$는 클래스 1의 샘플 비율입니다. 따라서 $p$값을 설정하여 샘플의 대다수가 한 개의 클래스에 속한 특성을 삭제할 수 있습니다.

이진 특성의 경우에도 variances_ 속성에서 분산을 확인할 수 있습니다.

In [7]:
thresholder.variances_

array([0.16, 0.16, 0.24])

VarianceThreshold 클래스는 수치 특성, 이진 특성에 상관없이 넘파이 var 함수를 사용하여 분산을 계산합니다. 다음 코드는 넘파이 var 함수를 사용하여 앞서 구한 분산을 동일하게 계산합니다.

In [8]:
import numpy as np
np.var(features, axis=0)

array([0.16, 0.16, 0.24])

이진 특성에 var 함수를 사용하는 것은 이진 특성일 때 베르누이 확률 변수의 분산과 같기 때문입니다. 분산 공식을 사용해 간단히 유도해볼 수 잇습니다. 먼저 분산 식을 다음과 같이 풀어 쓸 수 있습니다.
$$Var(x) = \frac{1}{n}\sum_{i=1}^n(x_i-\mu)^2 = \frac{1}{n}(\sum_{i=1}^n x_{i}^2-2\mu\sum_{i=1}^n x_{i}+n\mu^2)$$

0, 1로 이루어진 이진 특성일 경우 $x_{i}^2$은 $x_{i}$와 같으므로 $\frac{1}{n}$을 곱하면 첫 번째 항은 평균과 같아집니다. 두 번째 항도 마찬가지로 $\frac{1}{n}$을 곱하면 평균의 제곱으로 표현할 수 있습니다. 결국 다음과 같이 정리됩니다.
$$= \frac{1}{n}\sum_{i=1}^n x_{i} -2\mu\frac{1}{n}\sum_{i=1}^n x_{i} + \mu^2 = \mu - 2\mu^2 + \mu^2 = \mu - \mu^2 = \mu(1-\mu)$$

이진 특성의 평균 $\mu$는 클래스 1의 샘플 비율과 같습니다. 따라서 var 함수로 이진 특성의 분산을 계산하면 베르누이 확률 변수의 분산 $p(1-p)$와 같습니다. 

threshold 매개변수의 기본값은 0으로 모든 특성을 선택합니다.